In [8]:
!cat README.md

Sleep models pipeline

Requirements:


`sleep-models==1.1.1`

Instructions

This pipeline has 5 steps:

* 1: make_dataset. Creates a cached `anndata.AnnData` (.h5ad) only with cells from the desired background.
* These backgrounds are read from the config, and the annotation used is defined in the `backgrounds` folder inside `config["data_dir"]`

2: get_marker_genes: Iteratively removes markers genes with a fold change above an increasingly low threshold, so the cell types
in the background become as homogeneous as possible. A Dimensionality Reduction plot is generated at each threshold

3: remove_marker_genes: Set the genes with a threshold higher than the selected threshold to 0

4: train_models: have one or more of the supported models train using the transcriptomic data to predict sleep and wake

5: predict: predict the sleep / wake status of a cell from the same cell type, or a different one



You can adjust the settings of the pipeline by changing the config.yaml file in the `SleepML` folder


To change the logFC threshold that defines marker genes, update

* `user_defined_log2FC_threshold.KC` for KC
* `user_defined_log2FC_threshold.glia` for glia

To change the logFC threshold probled, update

* `log2FC_thresholds.KC` for KC
* `log2FC_thresholds.glia` for glia

To change the model architectures to be trained, update

* `arch`

To change the DR algorithms ran during the marker gene probing, update

* `DR_algorithm`

To change the number of *in-silico* replicates, update

* seeds list in `seeds`

To use the marker gene database downloaded from scope, set

* `markers`

In [ ]:
# Uncomment the one you wanna run
# !ln -sf config_wake_vs_sleep.yaml config.yaml 
# !ln -sf config_conditions.yaml config.yaml

**Please update the `results_dir` and `temp_data_dir` fields in the config so they have the timestamp of today**

In [1]:
!cat config.yaml

# Which background of cell types to work with
background:
  KC:
  glia:

# Marker gene filtering settings
user_defined_log2FC_threshold:
  # genes above this threshold will not be considerd for the analysis
  KC:
    1.0
  glia:
    1.0
  peptides:
    3.0
log2FC_thresholds:
  # A separate UMAP will be computed on the result of removing genes
  # with an abs log2FC higher than this
  KC:
    [8.0, 7.0, 6.0, 5.0, 4.0, 3.0, 2.5, 2.0, 1.5, 1.0, 0.5, 0.0]
  glia:
    [9.99, 9.0, 8.0, 7.0, 6.5, 6.0, 5.5, 5.0, 4.5, 4.0, 3.5, 3.0, 2.5, 2.0, 1.5, 1.0, 0.5, 0.0]
max_clusters:
  # if a marker is gene is shared by this or more cell types
  # of the background, it is still considered for the analysis
  # because it is not a marker anymore within the background
  KC:
    5
  glia:
    10

# Dataset generation settings
raw: True
exclude_genes_file: None
batch_genes_file: "batch_effects.xlsx"
highly_variable_genes: False
h5ad_input: "h5ad/Preloom/All_Combined_No_ZT2_Wake.h5ad"

# Model settings
arch:

In [2]:
! /home/vibflysleep/anaconda3/envs/SleepML/bin/python 01-pipeline/01_make_dataset.py

/home/vibflysleep/anaconda3/envs/SleepML/lib/python3.7/site-packages/anndata/compat/__init__.py:182: FutureWarning: Moving element from .uns['neighbors']['distances'] to .obsp['distances'].

This is where adjacency matrices should go now.
  FutureWarning,
/home/vibflysleep/anaconda3/envs/SleepML/lib/python3.7/site-packages/anndata/compat/__init__.py:182: FutureWarning: Moving element from .uns['neighbors']['connectivities'] to .obsp['connectivities'].

This is where adjacency matrices should go now.
  FutureWarning,
/home/vibflysleep/anaconda3/envs/SleepML/lib/python3.7/site-packages/pandas/core/arrays/categorical.py:2631: FutureWarning: The `inplace` parameter in pandas.Categorical.remove_unused_categories is deprecated and will be removed in a future version.
  res = method(*args, **kwargs)
/home/vibflysleep/anaconda3/envs/SleepML/lib/python3.7/site-packages/anndata/_core/anndata.py:1208: ImplicitModificationWarning: Initializing view as actual.
  "Initializing view as actual.", Impl

In [ ]:
! /home/vibflysleep/anaconda3/envs/SleepML/bin/python 01-pipeline/02_get_marker_genes.py

Done in 0.65 seconds
/Users/Antonio/Projects/SleepML/sleep-models/sleep_models/plotting/plotting.py:244: UserWarning: You have used the `textcoords` kwarg, but not the `xytext` kwarg.  This can lead to surprising results.
  size=22,
/Users/Antonio/Projects/SleepML/sleep-models/sleep_models/plotting/plotting.py:244: UserWarning: You have used the `textcoords` kwarg, but not the `xytext` kwarg.  This can lead to surprising results.
  size=22,
Computing DR at threshold = 8.0:   0%|                   | 0/12 [00:00<?, ?it/s]Done in 0.09 seconds
/Users/Antonio/Projects/SleepML/sleep-models/sleep_models/plotting/plotting.py:244: UserWarning: You have used the `textcoords` kwarg, but not the `xytext` kwarg.  This can lead to surprising results.
  size=22,
/Users/Antonio/Projects/SleepML/sleep-models/sleep_models/plotting/plotting.py:244: UserWarning: You have used the `textcoords` kwarg, but not the `xytext` kwarg.  This can lead to surprising results.
  size=22,
Computing DR at threshold = 7.

In [ ]:
! /home/vibflysleep/anaconda3/envs/SleepML/bin/python 01-pipeline/03_remove_marker_genes.py

In [ ]:
! /home/vibflysleep/anaconda3/envs/SleepML/bin/python 01-pipeline/04_train_models.py

In [ ]:
! /home/vibflysleep/anaconda3/envs/SleepML/bin/python 01-pipeline/05_predict.py